# Answering Business Questions using SQL

We are working for a company that sells and ships musical tracks to different countries. Customer can purchase single tracks from multiple albums or entire album in an invoice.

One day, the manager want us to show him:
- The number of tracks sold in the USA.
- The total dollar amount of sales assigned to each sales support agent.
- The total purchases from different countries.
- The percentage of individual tracks and whole album purchases
- The artist appears the most in playlists
- The number of tracks has been purchased and not

In this project, we will use the chinook dataset provided by Lerocha, which you can find in his [Github](https://github.com/lerocha/chinook-database). 
> Media related data was created using real data from an iTunes Library. Customer and employee information was manually created using fictitious names, addresses that can be located on Google maps, and other well formatted data (phone, fax, email, etc.). Sales information is auto generated using random data for a four year period.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Retrieve existing tables and views from connected database

In [2]:
%%sql
SELECT name, 
        type
FROM sqlite_master
WHERE type IN('table','view');

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Database schema diagram illustrates the structure of data tables
<img src='schema.png'>

## The number of tracks sold in the USA
The store is planning to add three out of fours albums from this list <br>

|Artist|Genre  |
|------|-----  |
|Regal |Hip-Hop|
|Red Tone|Punk|
|Meteor and the Girls|Pop|
|Slim Jim Bites|Blues|

The manager wants to know which are the top three genre among the four above.

Create a View to filter out purchases where billing_country is USA only

In [3]:
%%sql
CREATE VIEW usa_purchases AS
SELECT invoice_id
FROM invoice
WHERE billing_country = 'USA';

 * sqlite:///chinook.db
Done.


[]

The manager wants to know the number of track sold in USA:
- by genre
- showed in track unit and percentage

In [16]:
%%sql
SELECT g.name genre, SUM(il.quantity) number_of_tracks,
        SUM(il.quantity)*100/
                  (SELECT SUM(il.quantity)
                    FROM invoice_line il
                    JOIN usa_purchases pc 
                    ON il.invoice_id = pc.invoice_id) percentage
FROM genre g
JOIN track t ON g.genre_id = t.genre_id
JOIN invoice_line il ON il.track_id = t.track_id
JOIN usa_purchases pc ON pc.invoice_id = il.invoice_id
GROUP BY 1  
ORDER BY 2 DESC

 * sqlite:///chinook.db
Done.


genre,number_of_tracks,percentage
Rock,561,53
Alternative & Punk,130,12
Metal,124,11
R&B/Soul,53,5
Blues,36,3
Alternative,35,3
Pop,22,2
Latin,22,2
Hip Hop/Rap,20,1
Jazz,14,1


Hip Hop has the least sales of tracks being sold.<br>

Among four genres (Hip-Hop, Punk, Pop, Blues), the later three genres should be chosen to sell in the USA.<br>

## The total dollar amount of sales assigned to each sales support agent

Each customer get assigned to a sales support agent when they make the first purchase.

The manager wants to review performance of sales support agent. He asked us to provide sales data for all of them.

In [34]:
%%sql
WITH customers_purchases AS
    (SELECT i.customer_id, c.support_rep_id, SUM(i.total) total_purchases
    FROM invoice i JOIN customer c
    ON i.customer_id = c.customer_id
    GROUP BY 1)

SELECT e.first_name ||' '|| e.last_name agent,
        e.title, SUBSTR(e.hire_date,1,10) hire_date,
        ROUND(SUM(cp.total_purchases),1) sales,
        COUNT(cp.customer_id) num_of_customers
FROM employee e
JOIN customers_purchases cp ON cp.support_rep_id = e.employee_id
GROUP BY 1
HAVING e.title LIKE '%Agent%'
ORDER BY 4 DESC

 * sqlite:///chinook.db
Done.


agent,title,hire_date,sales,num_of_customers
Jane Peacock,Sales Support Agent,2017-04-01,1731.5,21
Margaret Park,Sales Support Agent,2017-05-03,1584.0,20
Steve Johnson,Sales Support Agent,2017-10-17,1393.9,18


From the above data, we can tell Jane has the most number of customers and total sales. Following by Margaret and Steve, respectively. <br>

The more customers a sale agent has, the higher sales agent had.<br>

To be fair, Steve joined the company 6 months after Jane, it makes sense when he came in last rank.

## The total purchases from different countries
The manager also wants to know following information based on countries:
- Total number of customers
- Total sales
- Average sales per customer
- Average order value

However, he wants all countries with only one customer are grouped into one group.

In [7]:
%%sql
WITH sales_by_country AS
(SELECT c.country, COUNT(DISTINCT(c.customer_id)) num_of_customers,
        ROUND(SUM(i.total),2) total_sales,
        ROUND(SUM(i.total)/COUNT(DISTINCT(c.customer_id)),2) avg_sales_per_customer,
        ROUND(SUM(i.total)/COUNT(i.invoice_id),2) avg_order_value,
        CASE  
            WHEN COUNT(DISTINCT(c.customer_id)) = 1 THEN 'Other'
            ELSE c.country
        END AS category,
        CASE
            WHEN COUNT(DISTINCT(c.customer_id)) = 1 THEN 1 ELSE 0 
        END AS sort
FROM customer c 
JOIN invoice i ON c.customer_id = i.customer_id
GROUP BY 1
ORDER BY 2 DESC)
SELECT category country, SUM(num_of_customers) num_of_customers, 
        SUM(total_sales) total_sales, 
        SUM(avg_sales_per_customer) avg_sales_per_customer,
        SUM(avg_order_value) avg_order_value
FROM sales_by_country
GROUP BY 1
ORDER BY sort, 3

 * sqlite:///chinook.db
Done.


country,num_of_customers,total_sales,avg_sales_per_customer,avg_order_value
India,2,183.15,91.58,8.72
Portugal,2,185.13,92.57,6.38
United Kingdom,3,245.52,81.84,8.77
Czech Republic,2,273.24,136.62,9.11
Germany,4,334.62,83.66,8.16
France,5,389.07,77.81,7.78
Brazil,5,427.68,85.54,7.01
Canada,8,535.59,66.95,7.05
USA,13,1040.49,80.04,7.94
Other,15,1094.94,1094.94,111.66


## The percentage of individual tracks and whole album purchases

Customers can make purchase of:
- A whole album
- A collection of one or more individual tracks.

The store doesn't let customer purchase the whole album and add additional individual tracks.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

The manager asked us the percentage of purchases that consists of individual tracks and album tracks.<br>

If the purchase is for whole album, all tracks in the invoice should be the same as all tracks in an album. To determine an album purchase: <br>
- Pick any track from the invoice
- Find its album
- Compare the tracks from album to tracks from invoice

In [8]:
%%sql
WITH invoice_album AS
(SELECT il.invoice_id, il.track_id, t.album_id
FROM invoice_line il
JOIN track t ON il.track_id = t.track_id
GROUP BY 1)
SELECT album_purchase, COUNT(invoice_id) number_of_invoices,
        ROUND(CAST(COUNT(invoice_id) AS FLOAT)/
              (SELECT COUNT(invoice_id)
               FROM invoice)*100,2) percentage_of_invoices
FROM
(SELECT invoice_id, 
    CASE
        WHEN 
            (SELECT il.track_id
             FROM invoice_line il
             WHERE il.invoice_id = ia.invoice_id
        EXCEPT
            SELECT t.track_id
            FROM track t
            WHERE t.album_id = ia.album_id) IS NULL
        AND
            (SELECT t.track_id
            FROM track t
            WHERE t.album_id = ia.album_id 
        EXCEPT
            SELECT il.track_id
            FROM invoice_line il
            WHERE il.invoice_id = ia.invoice_id) IS NULL
        THEN 'yes' ELSE 'no' END AS 'album_purchase'
FROM invoice_album ia)
GROUP BY 1

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percentage_of_invoices
no,500,81.43
yes,114,18.57


Out of 614 purchases, there are 114 whole-album purchases.<br>

Album purchase makes up approximately 19% of purchases.

## The number of tracks have been purchased vs not purchased
The manager wants to understand customers' demand better that he asked us to provide the number of tracks that have been purchased or not.

In [56]:
%%sql
SELECT COUNT(*) FROM track

 * sqlite:///chinook.db
Done.


COUNT(*)
3503


In [69]:
%%sql
WITH track_purchases AS
(SELECT track_id, 
    CASE WHEN track_id IN (SELECT DISTINCT(track_id)
                          FROM invoice_line)
    THEN 'yes' ELSE 'no'
    END AS purchased
FROM track)

SELECT purchased, COUNT(*) num_of_tracks
FROM track_purchases
GROUP BY purchased

 * sqlite:///chinook.db
Done.


purchased,num_of_tracks
no,1697
yes,1806


The store has almost 50% of track inventory has not been sold. We need to come up with a promotion to boost sales for these tracks. <br>

Let's take a look into which genre the store still have the most of.

In [107]:
%%sql
WITH tracks_purchases AS
(SELECT t.track_id, g.name genre, 
    CASE WHEN track_id IN (SELECT DISTINCT(track_id)
                          FROM invoice_line)
    THEN 'yes' ELSE 'no'
    END AS purchased
FROM track t
JOIN genre g ON t.genre_id = g.genre_id)

SELECT tp.genre, COUNT(*) num_of_unsold_tracks
FROM tracks_purchases tp
JOIN track t ON t.track_id = tp.track_id
GROUP BY 1
HAVING purchased = 'no'
ORDER BY 2 DESC

 * sqlite:///chinook.db
Done.


genre,num_of_unsold_tracks
Latin,579
Alternative & Punk,332
TV Shows,93
Drama,64
Reggae,58
Pop,48
Soundtrack,43
Hip Hop/Rap,35
World,28
Heavy Metal,28


One third of the inventory belongs to Latin genre. <br>
The next genre with high inventory is Alternative & Punk.<br>

## The artist is used in the most playlists

In [53]:
%%sql
SELECT artist_name, COUNT(DISTINCT(playlist_id)) num_of_playlists
FROM
    (SELECT pt.playlist_id, pt.track_id, at.name artist_name
    FROM playlist_track pt
    JOIN track t ON t.track_id = pt.track_id
    JOIN album ab ON ab.album_id = t.album_id
    JOIN artist at ON at.artist_id = ab.artist_id)
GROUP BY 1
ORDER BY 2 DESC
LIMIT 6;

 * sqlite:///chinook.db
Done.


artist_name,num_of_playlists
Eugene Ormandy,7
The King's Singers,6
English Concert & Trevor Pinnock,6
Berliner Philharmoniker & Herbert Von Karajan,6
Academy of St. Martin in the Fields & Sir Neville Marriner,6
Yo-Yo Ma,5


Eugene Ormandy is used in 7 playlists.

## Conclusion
From the analysis above, we found that:
#### Country
- USA leads with total sales of 1,040 dollars from 13 customers.
- The second biggest market is Canada with 8 customers and 536 dollars.

#### Genre
- Rock is the most popular genre in the U.S, which makes up 53% of sales.
- Next in line are Alternative & Punk, Metal, R&B/Soul with 12%, 11%, 5% respectively.

#### Purchase type
- Customers tend to buy individual tracks four times more than whole album. The management can go forward with the new strategy.

#### Sales by Agent
- The more customers a sale agent has, the higher sales that agent had.

#### Inventory
- Approximately 1,700 tracks still have not been sold.
- One third of that is in Latin genre

#### Artist
- Eugene Ormandy appears in 7 playlists.